## Importación de Librerías
Importamos la lista de librerías que vamos a utilizar, y que ya están listadas en otro archivo:

In [126]:
%run ".\\00. Importación de Librerías\\Librerias_para_importar.ipynb"

## Definición de funciones
Importamos las funciones definidas en otro archivo:

In [127]:
%run ".\\00. Definición de funciones\\Funciones_definidas.ipynb"

## Estructuración y Limpieza de los Datos:

In [128]:
data = pd.read_csv(".\\0. Datos de Entrada\\Datos_Completos.csv")

# Renombramos las columnas:
data = data.rename(columns={'fecha': "fecha_IOL",'apertura': "open",'cierre': "close",'máximo': "high", 'mínimo': "low" }) 
agregado_al_nombre_del_archivo = ''

In [ ]:
# TRATAMIENTO DE LOS DÍAS CON TODAS SUS COTIZACIONES NULAS ( = 0 ):

data["sin_cotizacion"] = np.where( (data["open"]==0) & (data["close"]==0) & (data["high"]==0) & (data["low"]==0) , 1, 0)
print("La cantidad de filas que tienen todas sus cotizaciones nulas es:", data[data.sin_cotizacion == 1].shape[0], '\n' )
data = data[data.sin_cotizacion == 0]  # Aquí se eliminan las filas que tienen cotizaciones nulas.
data = data.drop(["sin_cotizacion"], axis=1)


# TRATAMIENTO DE LOS DÍAS CON COTIZACIONES DE CLOSE o OPEN NULAS ( = 0 ):

data["sin_cotizacion"] = np.where( ((data["open"]==0) | (data["close"]==0)), 1, 0)
print("La cantidad de filas que tienen cotizaciones de Close o Open nulas es:", data[data.sin_cotizacion == 1].shape[0], '\n' )
data = data[data.sin_cotizacion == 0]  # Aquí se eliminan las filas que tienen cotizaciones nulas.
data = data.drop(["sin_cotizacion"], axis=1)


# TRATAMIENTO DE LOS VALORES de VOLUMEN NULO (= 0) (Utilizar sólo uno de los siguientes métodos):
print("La cantidad de filas que tienen Volumen(t) nulo es:", data[data.volumen == 0].shape[0] )

# # Eliminamos aquellas filas que tienen Volumen igual a cero (muchas veces corresponden a días feriados o similares, salvo cuando hay errores en la
# # base de datos (como el caso de CEPU) o cuando la acción no tiene transacciones pero se trata de un día hábil):
# data = data[data.volumen != 0] 

# En aquellas filas donde el Volumen es cero, se reemplaza el cero por un número 'muy bajo':
numero_muy_bajo = 1000
data["volumen"].replace(0, numero_muy_bajo, inplace=True)


In [ ]:
# Eliminamos las filas repetidas:

print("La cantidad de filas repetidas es:", len(data[data.duplicated()].index) )
data = data.drop(data[data.duplicated()].index, axis=0)

In [ ]:
# Eliminamos todas las filas donde para una determinada acción exista más de 1 fila con la misma fecha:

data["fecha_aux"] = list(map(lambda x: x[:10], data["fecha_IOL"]))
data["concatenado"] = data["titulo"] + "--" + data["fecha_aux"]
concatenados_repetidos = list(data["concatenado"].value_counts() [data["concatenado"].value_counts() != 1].index)
indices_a_eliminar = data [data["concatenado"].isin (concatenados_repetidos)].index

if len(concatenados_repetidos) != 0:
    for fecha, cantidad in data.loc[indices_a_eliminar] ["fecha_aux"].value_counts().items():
        print("Se eliminaron ", cantidad, " filas repetidas con la fecha:", fecha)
else:
    print("Se encontraron  0  filas con la fecha repetida para un mismo título.")

data = data.drop (indices_a_eliminar, axis=0)   # Acá eliminamos las filas.

data = data.drop (['fecha_aux'], axis=1)
data = data.drop (['concatenado'], axis=1)

In [132]:
# Formateamos la Fecha para que se muestre con el formato correcto:

fecha = []
año = []
mes = []
dia = []

for i in data.fecha_IOL:
    año.append(i[0:4])
    mes.append(i[5:7])
    dia.append(i[8:10])
    fecha.append(i[0:i.find("T")])
    
data ["año"] = [int(i) for i in año]
data ["mes"] = [int(i) for i in mes]  
data ["dia"] = [int(i) for i in dia] 
data ["fecha"] = pd.to_datetime(fecha, dayfirst=True)
data["dia_de_semana"] = [i.weekday() + 1 for i in data.fecha]  # Asignamos el 1 a los días Lunes y el 5 a los días Viernes.

data.fecha_IOL = pd.to_datetime(data.fecha_IOL, dayfirst=True)

In [133]:
# TRATAMIENTO DE LOS ERRORES DE COTIZACIÓN EN LA BASE DE DATOS:

# - Cuando High(t) = 0  ó  cuando Low(t) = 0 :

print("La cantidad de filas que tienen High(t) nulo es:", data[data.high == 0].shape[0] )
print("La cantidad de filas que tienen Low(t) nulo es:", data[data.low == 0].shape[0], '\n' )

lista_de_df = []
for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 

  # Se reemplaza los valores iguales a cero (0) de High(t) y de Low(t) por su valor anterior:
  df_aux["high"].replace(to_replace=0, method='ffill', inplace=True)
  df_aux["low"].replace(to_replace=0, method='ffill', inplace=True)

  lista_de_df.append(df_aux)

data = pd.concat(lista_de_df, axis=0)
data = data.sort_index()



# - Cuando High(t) < Open(t)  ó  cuando High(t) < Close(t)  ó  cuando Low(t) > Close(t)  ó  cuando Low(t) > Open(t):

data["error_en_High_tipo_1"] = np.where( data["high"] < data["open"]  , 1, 0)
data["error_en_High_tipo_2"] = np.where( data["high"] < data["close"]  , 1, 0)
data["error_en_Low_tipo_1"] = np.where( data["low"] > data["close"]  , 1, 0)
data["error_en_Low_tipo_2"] = np.where( data["low"] > data["open"]  , 1, 0)

print("La cantidad de filas que tiene el error en que High(t) < Open(t) es:", data[data["error_en_High_tipo_1"] == 1].shape[0] )
print("La cantidad de filas que tiene el error en que High(t) < Close(t) es:", data[data["error_en_High_tipo_2"] == 1].shape[0] )
print("La cantidad de filas que tiene el error en que Low(t) > Close(t) es:", data[data["error_en_Low_tipo_1"] == 1].shape[0] )
print("La cantidad de filas que tiene el error en que Low(t) > Open(t) es:", data[data["error_en_Low_tipo_2"] == 1].shape[0] )

data["high"] = np.where( data["error_en_High_tipo_1"] == 1 , data["open"], data["high"] )
data["high"] = np.where( data["error_en_High_tipo_2"] == 1 , data["close"], data["high"] )
data["low"] = np.where( data["error_en_Low_tipo_1"] == 1 , data["close"], data["low"] )
data["low"] = np.where( data["error_en_Low_tipo_2"] == 1 , data["open"], data["low"] )

data = data.drop(["error_en_High_tipo_1"], axis=1)
data = data.drop(["error_en_High_tipo_2"], axis=1)
data = data.drop(["error_en_Low_tipo_1"], axis=1)
data = data.drop(["error_en_Low_tipo_2"], axis=1)

La cantidad de filas que tienen High(t) nulo es: 3
La cantidad de filas que tienen Low(t) nulo es: 3 

La cantidad de filas que tiene el error en que High(t) < Open(t) es: 0
La cantidad de filas que tiene el error en que High(t) < Close(t) es: 39
La cantidad de filas que tiene el error en que Low(t) > Close(t) es: 25
La cantidad de filas que tiene el error en que Low(t) > Open(t) es: 0


## Agregado de Features Externos al Merval (variables macroeconómicas, etc)

In [134]:
# Cargamos los datos del Dólar CCL 

dolar_ccl = pd.read_excel(".\\0. Datos de Entrada\\dolar_ccl.xlsx")
dolar_ccl.sort_values( by=["fecha"], ascending = True, inplace=True )   # Ordenamos por 'fecha' de la más antigua a la más reciente.
lista_completa_de_fechas = pd.date_range( min(data.fecha), max(data.fecha) )  # Creamos un listado con todas las fechas que utilizaremos.
df_fechas_completas = pd.DataFrame( {"fecha": lista_completa_de_fechas} )
dolar_ccl = df_fechas_completas.merge( dolar_ccl, on = "fecha", how = 'left' )
dolar_ccl = dolar_ccl.fillna(method = 'ffill')  # Completamos los NaN con los valores del dólar del día anterior.
dolar_ccl = dolar_ccl.fillna(method = 'bfill')  # Completamos los valores de las primeras fechas para los cuales no tenemos datos con los valores del dólar del día siguiente.

dolar_ccl["valor_promedio"] =  np.where(dolar_ccl["valor_de_venta"] == 0, dolar_ccl["valor_de_compra"],
                                           dolar_ccl[['valor_de_compra','valor_de_venta']].mean(axis=1))

# Eliminamos las fechas repetidas:
for fecha_repetida in (dolar_ccl.fecha.value_counts()[ dolar_ccl.fecha.value_counts() >= 2 ]).index :
    lista_de_indices_a_eliminar = dolar_ccl [dolar_ccl.fecha == fecha_repetida].index[ 0 : (len(dolar_ccl [dolar_ccl.fecha == fecha_repetida]) -1 ) ]
    dolar_ccl = dolar_ccl.drop( lista_de_indices_a_eliminar, axis=0)
dolar_ccl = dolar_ccl.reset_index(drop = True)

# Agregamos los datos al DataFrame 'data' :
data = data.merge( dolar_ccl[["fecha", "valor_promedio", "valor_de_compra", "valor_de_venta" ]], on = "fecha", how = 'left' )
data.rename(columns = {'valor_promedio':'dolar_ccl', 'valor_de_compra':'dolar_ccl_compra', 'valor_de_venta':'dolar_ccl_venta',}, inplace = True)   # Le cambiamos el nombre a las columnas.


In [135]:
# Cargamos los datos del Dólar Blue

dolar_blue = pd.read_excel(".\\0. Datos de Entrada\\dolar_blue.xlsx")
dolar_blue.rename(columns={'Fecha':'fecha', 'Compra':'valor_de_compra', 'Venta':'valor_de_venta'}, inplace=True)
dolar_blue.sort_values( by=["fecha"], ascending = True, inplace=True )   # Ordenamos por 'fecha' de la más antigua a la más reciente.
lista_completa_de_fechas = pd.date_range( min(data.fecha), max(data.fecha) )  # Creamos un listado con todas las fechas que utilizaremos.
df_fechas_completas = pd.DataFrame( {"fecha": lista_completa_de_fechas} )
dolar_blue = df_fechas_completas.merge( dolar_blue, on = "fecha", how = 'left' )
dolar_blue = dolar_blue.fillna(method = 'ffill')  # Completamos los NaN con los valores del dólar del día anterior.
dolar_blue = dolar_blue.fillna(method = 'bfill')  # Completamos los valores de las primeras fechas para los cuales no tenemos datos con los valores del dólar del día siguiente.

dolar_blue["valor_promedio"] = dolar_blue[['valor_de_compra','valor_de_venta']].mean(axis=1)

# Eliminamos las fechas repetidas:
for fecha_repetida in (dolar_blue.fecha.value_counts()[ dolar_blue.fecha.value_counts() >= 2 ]).index :
    lista_de_indices_a_eliminar = dolar_blue [dolar_blue.fecha == fecha_repetida].index[ 0 : (len(dolar_blue [dolar_blue.fecha == fecha_repetida]) -1 ) ]
    dolar_blue = dolar_blue.drop( lista_de_indices_a_eliminar, axis=0)
dolar_blue = dolar_blue.reset_index(drop = True)

# Agregamos los datos al DataFrame 'data' :
data = data.merge( dolar_blue[["fecha", "valor_promedio", "valor_de_compra", "valor_de_venta" ]], on = "fecha", how = 'left' )
data.rename(columns = {'valor_promedio':'dolar_blue', 'valor_de_compra':'dolar_blue_compra', 'valor_de_venta':'dolar_blue_venta',}, inplace = True)   # Le cambiamos el nombre a las columnas.


In [138]:
# Creamos para cada acción/título las variables de entrada 'derivadas' a partir de los datos externos (macroeconómicos, etc):

lista_de_df = []

for accion in data.groupby(['titulo']).groups.keys():
    df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 


    df_aux ["dolar_blue(t-1)/dolar_blue(t-2)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(2)
    df_aux ["dolar_blue(t-1)/dolar_blue(t-3)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(3)
    df_aux ["dolar_blue(t-1)/dolar_blue(t-5)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(5)
    df_aux ["dolar_blue(t-1)/dolar_blue(t-10)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(10)
    df_aux ["dolar_blue(t-1)/dolar_blue(t-20)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(20)
    df_aux ["dolar_blue(t-1)/dolar_blue(t-50)"] = df_aux ["dolar_blue"].shift(1) / df_aux ["dolar_blue"].shift(50)

    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-2)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(2)
    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-3)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(3)
    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-5)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(5)
    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-10)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(10)
    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-20)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(20)
    df_aux ["dolar_ccl(t-1)/dolar_ccl(t-50)"] = df_aux ["dolar_ccl"].shift(1) / df_aux ["dolar_ccl"].shift(50)

    df_aux ["spread_dolar_blue(t-1)"] = (df_aux ["dolar_blue_venta"].shift(1) - df_aux ["dolar_blue_compra"].shift(1)) / df_aux ["dolar_blue"].shift(1)
    df_aux ["spread_dolar_ccl(t-1)"] = (df_aux ["dolar_ccl_venta"].shift(1) - df_aux ["dolar_ccl_compra"].shift(1)) / df_aux ["dolar_ccl"].shift(1)
    df_aux ["spread_dolar_ccl(t-1)"] = np.where( df_aux ["spread_dolar_ccl(t-1)"] == - 1, 0, df_aux ["spread_dolar_ccl(t-1)"])
    
    
    # EMA (Exponencial Móvil Simple): 

    df_aux['EMA_dolar_ccl_k=3(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,3)
    df_aux['EMA_dolar_ccl_k=10(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,10)
    df_aux['EMA_dolar_ccl_k=5(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,5)
    df_aux['EMA_dolar_ccl_k=2(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,2)
    df_aux['EMA_dolar_ccl_k=15(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,15)
    df_aux['EMA_dolar_ccl_k=20(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,20)
    df_aux['EMA_dolar_ccl_k=25(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,25)
    df_aux['EMA_dolar_ccl_k=30(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,30)
    df_aux['EMA_dolar_ccl_k=40(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,40)
    df_aux['EMA_dolar_ccl_k=50(t)'] = ta.trend.ema_indicator(df_aux.dolar_ccl,50)

    df_aux['EMA_dolar_blue_k=3(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,3)
    df_aux['EMA_dolar_blue_k=10(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,10)
    df_aux['EMA_dolar_blue_k=5(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,5)
    df_aux['EMA_dolar_blue_k=2(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,2)
    df_aux['EMA_dolar_blue_k=15(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,15)
    df_aux['EMA_dolar_blue_k=20(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,20)
    df_aux['EMA_dolar_blue_k=25(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,25)
    df_aux['EMA_dolar_blue_k=30(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,30)
    df_aux['EMA_dolar_blue_k=40(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,40)
    df_aux['EMA_dolar_blue_k=50(t)'] = ta.trend.ema_indicator(df_aux.dolar_blue,50)

    df_aux ["EMA_dolar_blue_k=5(t-1)/EMA_dolar_blue_k=5(t-2)"] = df_aux ["EMA_dolar_blue_k=5(t)"].shift(1) / df_aux ["EMA_dolar_blue_k=5(t)"].shift(2)
    df_aux ["EMA_dolar_blue_k=5(t-1)/dolar_blue(t-1)"] = df_aux ["EMA_dolar_blue_k=5(t)"].shift(1) / df_aux ["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=5(t-3)/dolar_blue(t-1)"] = df_aux ["EMA_dolar_blue_k=5(t)"].shift(3) / df_aux ["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=5(t-5)/dolar_blue(t-1)"] = df_aux ["EMA_dolar_blue_k=5(t)"].shift(5) / df_aux ["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=3(t-1)/EMA_dolar_blue_k=3(t-2)"] = df_aux ["EMA_dolar_blue_k=3(t)"].shift(1) / df_aux ["EMA_dolar_blue_k=3(t)"].shift(2)
    df_aux ["EMA_dolar_blue_k=10(t-1)/EMA_dolar_blue_k=10(t-2)"] = df_aux ["EMA_dolar_blue_k=10(t)"].shift(1) / df_aux ["EMA_dolar_blue_k=10(t)"].shift(2)
    df_aux ["EMA_dolar_blue_k=40(t-1)/dolar_blue(t-1)"] = df_aux["EMA_dolar_blue_k=40(t)"].shift(1) / df_aux["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=30(t-1)/dolar_blue(t-1)"] = df_aux["EMA_dolar_blue_k=30(t)"].shift(1) / df_aux["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=15(t-1)/dolar_blue(t-1)"] = df_aux["EMA_dolar_blue_k=15(t)"].shift(1) / df_aux["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=20(t-1)/dolar_blue(t-1)"] = df_aux["EMA_dolar_blue_k=20(t)"].shift(1) / df_aux["dolar_blue"].shift(1)
    df_aux ["EMA_dolar_blue_k=10(t-1)/dolar_blue(t-1)"] = df_aux["EMA_dolar_blue_k=10(t)"].shift(1) / df_aux["dolar_blue"].shift(1)

    df_aux ["EMA_dolar_ccl_k=5(t-1)/EMA_dolar_ccl_k=5(t-2)"] = df_aux ["EMA_dolar_ccl_k=5(t)"].shift(1) / df_aux ["EMA_dolar_ccl_k=5(t)"].shift(2)
    df_aux ["EMA_dolar_ccl_k=5(t-1)/dolar_ccl(t-1)"] = df_aux ["EMA_dolar_ccl_k=5(t)"].shift(1) / df_aux ["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=5(t-3)/dolar_ccl(t-1)"] = df_aux ["EMA_dolar_ccl_k=5(t)"].shift(3) / df_aux ["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=5(t-5)/dolar_ccl(t-1)"] = df_aux ["EMA_dolar_ccl_k=5(t)"].shift(5) / df_aux ["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=3(t-1)/EMA_dolar_ccl_k=3(t-2)"] = df_aux ["EMA_dolar_ccl_k=3(t)"].shift(1) / df_aux ["EMA_dolar_ccl_k=3(t)"].shift(2)
    df_aux ["EMA_dolar_ccl_k=10(t-1)/EMA_dolar_ccl_k=10(t-2)"] = df_aux ["EMA_dolar_ccl_k=10(t)"].shift(1) / df_aux ["EMA_dolar_ccl_k=10(t)"].shift(2)
    df_aux ["EMA_dolar_ccl_k=40(t-1)/dolar_ccl(t-1)"] = df_aux["EMA_dolar_ccl_k=40(t)"].shift(1) / df_aux["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=30(t-1)/dolar_ccl(t-1)"] = df_aux["EMA_dolar_ccl_k=30(t)"].shift(1) / df_aux["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=15(t-1)/dolar_ccl(t-1)"] = df_aux["EMA_dolar_ccl_k=15(t)"].shift(1) / df_aux["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=20(t-1)/dolar_ccl(t-1)"] = df_aux["EMA_dolar_ccl_k=20(t)"].shift(1) / df_aux["dolar_ccl"].shift(1)
    df_aux ["EMA_dolar_ccl_k=10(t-1)/dolar_ccl(t-1)"] = df_aux["EMA_dolar_ccl_k=10(t)"].shift(1) / df_aux["dolar_ccl"].shift(1)


    lista_de_df.append(df_aux)

data = pd.concat(lista_de_df, axis=0)



## Creación de Features:

In [ ]:
# Creamos para cada acción/título las variables de entrada 'derivadas' a partir de los datos básicos:

tiempo_inicial = time()   # Para medir el tiempo

lista_de_df = []
cantidad_dias = 2  # Definimos la cantidad de días hacia atrás que el algoritmo va a "ver" qué sucedió con los precios, dado un determinado día.

for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 


  # VARIABLES RELACIONADAS ENTRE: Open - Close - High - Low : 

  df_aux ["precio_medio"] = (df_aux ["low"] + df_aux ["high"]) / 2

  df_aux ["open(t)/open(t-1)"] = df_aux ["open"].shift(0) / df_aux ["open"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["open(t-"+str(i+1)+")/open(t-"+str(i+2)+")"] = df_aux ["open"].shift(i+1) / df_aux ["open"].shift(i+2)

  df_aux ["close(t)/close(t-1)"] = df_aux ["close"].shift(0) / df_aux ["close"].shift(1)
  df_aux ["close(t-1)/close(t-5)"] = df_aux ["close"].shift(1) / df_aux ["close"].shift(5)
  df_aux ["close(t-1)/close(t-10)"] = df_aux ["close"].shift(1) / df_aux ["close"].shift(10)
  df_aux ["close(t-1)/close(t-20)"] = df_aux ["close"].shift(1) / df_aux ["close"].shift(20)
  df_aux ["close(t-1)/close(t-50)"] = df_aux ["close"].shift(1) / df_aux ["close"].shift(50)
  for i in range(cantidad_dias):
    df_aux ["close(t-"+str(i+1)+")/close(t-"+str(i+2)+")"] = df_aux ["close"].shift(i+1) / df_aux ["close"].shift(i+2)

  df_aux ["high(t)/high(t-1)"] = df_aux ["high"].shift(0) / df_aux ["high"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["high(t-"+str(i+1)+")/high(t-"+str(i+2)+")"] = df_aux ["high"].shift(i+1) / df_aux ["high"].shift(i+2)

  df_aux ["low(t)/low(t-1)"] = df_aux ["low"].shift(0) / df_aux ["low"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["low(t-"+str(i+1)+")/low(t-"+str(i+2)+")"] = df_aux ["low"].shift(i+1) / df_aux ["low"].shift(i+2)

  df_aux ["volumen(t)/volumen(t-1)"] = df_aux ["volumen"].shift(0) / df_aux ["volumen"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["volumen(t-"+str(i+1)+")/volumen(t-"+str(i+2)+")"] = df_aux ["volumen"].shift(i+1) / df_aux ["volumen"].shift(i+2)


  
  df_aux ["high(t)/open(t)"] = (df_aux.high / df_aux.open) 
  for i in range(cantidad_dias):
    df_aux ["high(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["high(t)/open(t)"].shift(i+1)

  df_aux ["open(t)/high(t-1)"] = df_aux ["open"].shift(0) / df_aux ["high"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["open(t-"+str(i+1)+")/high(t-"+str(i+2)+")"] = df_aux ["open"].shift(i+1) / df_aux ["high"].shift(i+2)
    
  df_aux ["close(t)/open(t)"] = (df_aux.close / df_aux.open) 
  for i in range(cantidad_dias):
    df_aux ["close(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["close(t)/open(t)"].shift(i+1)

  df_aux ["open(t)/close(t-1)"] = df_aux ["open"].shift(0) / df_aux ["close"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["open(t-"+str(i+1)+")/close(t-"+str(i+2)+")"] = df_aux ["open"].shift(i+1) / df_aux ["close"].shift(i+2)

  df_aux ["low(t)/open(t)"] = (df_aux.low / df_aux.open) 
  for i in range(cantidad_dias):
    df_aux ["low(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["low(t)/open(t)"].shift(i+1)

  df_aux ["open(t)/low(t-1)"] = df_aux ["open"].shift(0) / df_aux ["low"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["open(t-"+str(i+1)+")/low(t-"+str(i+2)+")"] = df_aux ["open"].shift(i+1) / df_aux ["low"].shift(i+2)

  df_aux ["high(t)/low(t)"] = (df_aux.high / df_aux.low) 
  for i in range(cantidad_dias):
    df_aux ["high(t-"+str(i+1)+")/low(t-"+str(i+1)+")"] = df_aux ["high(t)/low(t)"].shift(i+1)

  df_aux ["high(t)/low(t-1)"] = df_aux ["high"].shift(0) / df_aux ["low"].shift(1)
  for i in range(5):
    df_aux ["high(t-"+str(i+1)+")/low(t-"+str(i+2)+")"] = df_aux ["high"].shift(i+1) / df_aux ["low"].shift(i+2)

  df_aux ["high(t)/close(t)"] = (df_aux.high / df_aux.close) 
  for i in range(cantidad_dias):
    df_aux ["high(t-"+str(i+1)+")/close(t-"+str(i+1)+")"] = df_aux ["high(t)/close(t)"].shift(i+1)

  df_aux ["high(t)/close(t-1)"] = df_aux ["high"].shift(0) / df_aux ["close"].shift(1)
  for i in range(cantidad_dias):
    df_aux ["high(t-"+str(i+1)+")/close(t-"+str(i+2)+")"] = df_aux ["high"].shift(i+1) / df_aux ["close"].shift(i+2)



  # SMA (Media Móvil Simple): 

  # df_aux['SMA_close_k=5(t)'] = ta.trend.sma_indicator(df_aux.close,5)
  # df_aux ["SMA_close_k=5(t-1)/SMA_close_k=5(t-2)"] = df_aux ["SMA_close_k=5(t)"].shift(1) / df_aux ["SMA_close_k=5(t)"].shift(2)
  # df_aux ["SMA_close_k=5(t-1)/close(t-1)"] = df_aux ["SMA_close_k=5(t)"].shift(1) / df_aux ["close"].shift(1)
  # df_aux ["SMA_close_k=5(t-3)/close(t-1)"] = df_aux ["SMA_close_k=5(t)"].shift(3) / df_aux ["close"].shift(1)
  # df_aux ["SMA_close_k=5(t-5)/close(t-1)"] = df_aux ["SMA_close_k=5(t)"].shift(5) / df_aux ["close"].shift(1)

  # df_aux['SMA_close_k=3(t)'] = ta.trend.sma_indicator(df_aux.close,3)
  # df_aux ["SMA_close_k=3(t-1)/SMA_close_k=3(t-2)"] = df_aux ["SMA_close_k=3(t)"].shift(1) / df_aux ["SMA_close_k=3(t)"].shift(2)

  # df_aux['SMA_close_k=10(t)'] = ta.trend.sma_indicator(df_aux.close,10)
  # df_aux ["SMA_close_k=10(t-1)/SMA_close_k=10(t-2)"] = df_aux ["SMA_close_k=10(t)"].shift(1) / df_aux ["SMA_close_k=10(t)"].shift(2)


  # EMA (Exponencial Móvil Simple): 

  df_aux['EMA_open_k=5(t)'] = ta.trend.ema_indicator(df_aux.open,5)
  df_aux['EMA_open_k=6(t)'] = ta.trend.ema_indicator(df_aux.open,6)
  df_aux['EMA_open_k=10(t)'] = ta.trend.ema_indicator(df_aux.open,10)
  df_aux['EMA_open_k=20(t)'] = ta.trend.ema_indicator(df_aux.open,20)

  df_aux['EMA_low_k=5(t)'] = ta.trend.ema_indicator(df_aux.low,5)
  df_aux['EMA_low_k=10(t)'] = ta.trend.ema_indicator(df_aux.low,10)
  df_aux['EMA_low_k=20(t)'] = ta.trend.ema_indicator(df_aux.low,20)
  df_aux['EMA_low_k=3(t)'] = ta.trend.ema_indicator(df_aux.low,3)

  df_aux['EMA_close_k=3(t)'] = ta.trend.ema_indicator(df_aux.close,3)
  df_aux['EMA_close_k=10(t)'] = ta.trend.ema_indicator(df_aux.close,10)
  df_aux['EMA_close_k=5(t)'] = ta.trend.ema_indicator(df_aux.close,5)
  df_aux['EMA_close_k=2(t)'] = ta.trend.ema_indicator(df_aux.close,2)
  df_aux['EMA_close_k=15(t)'] = ta.trend.ema_indicator(df_aux.close,15)
  df_aux['EMA_close_k=20(t)'] = ta.trend.ema_indicator(df_aux.close,20)
  df_aux['EMA_close_k=25(t)'] = ta.trend.ema_indicator(df_aux.close,25)
  df_aux['EMA_close_k=30(t)'] = ta.trend.ema_indicator(df_aux.close,30)
  df_aux['EMA_close_k=40(t)'] = ta.trend.ema_indicator(df_aux.close,40)
  df_aux['EMA_close_k=50(t)'] = ta.trend.ema_indicator(df_aux.close,50)

  df_aux['EMA_high_k=2(t)'] = ta.trend.ema_indicator(df_aux.high,2)
  df_aux['EMA_high_k=3(t)'] = ta.trend.ema_indicator(df_aux.high,3)
  df_aux['EMA_high_k=5(t)'] = ta.trend.ema_indicator(df_aux.high,5)
  df_aux['EMA_high_k=10(t)'] = ta.trend.ema_indicator(df_aux.high,10)
  df_aux['EMA_high_k=20(t)'] = ta.trend.ema_indicator(df_aux.high,20)


  df_aux ["EMA_close_k=5(t-1)/EMA_close_k=5(t-2)"] = df_aux ["EMA_close_k=5(t)"].shift(1) / df_aux ["EMA_close_k=5(t)"].shift(2)
  df_aux ["EMA_close_k=5(t-1)/close(t-1)"] = df_aux ["EMA_close_k=5(t)"].shift(1) / df_aux ["close"].shift(1)
  df_aux ["EMA_close_k=5(t-3)/close(t-1)"] = df_aux ["EMA_close_k=5(t)"].shift(3) / df_aux ["close"].shift(1)
  df_aux ["EMA_close_k=5(t-5)/close(t-1)"] = df_aux ["EMA_close_k=5(t)"].shift(5) / df_aux ["close"].shift(1)
  df_aux ["EMA_close_k=3(t-1)/EMA_close_k=3(t-2)"] = df_aux ["EMA_close_k=3(t)"].shift(1) / df_aux ["EMA_close_k=3(t)"].shift(2)
  df_aux ["EMA_close_k=10(t-1)/EMA_close_k=10(t-2)"] = df_aux ["EMA_close_k=10(t)"].shift(1) / df_aux ["EMA_close_k=10(t)"].shift(2)
  df_aux ["EMA_close_k=40(t-1)/precio_medio(t-1)"] = df_aux["EMA_close_k=40(t)"].shift(1) / df_aux["precio_medio"].shift(1)
  df_aux ["EMA_close_k=30(t-1)/precio_medio(t-1)"] = df_aux["EMA_close_k=30(t)"].shift(1) / df_aux["precio_medio"].shift(1)
  df_aux ["EMA_close_k=15(t-1)/precio_medio(t-1)"] = df_aux["EMA_close_k=15(t)"].shift(1) / df_aux["precio_medio"].shift(1)
  df_aux ["EMA_close_k=20(t-1)/precio_medio(t-1)"] = df_aux["EMA_close_k=20(t)"].shift(1) / df_aux["precio_medio"].shift(1)
  df_aux ["EMA_close_k=10(t-1)/precio_medio(t-1)"] = df_aux["EMA_close_k=10(t)"].shift(1) / df_aux["precio_medio"].shift(1)
  df_aux ["EMA_close_k=20(t-1)/EMA_close_k=5(t-1)"] = df_aux["EMA_close_k=20(t)"].shift(1) / df_aux["EMA_close_k=5(t)"].shift(1)
  df_aux ["(EMA_close_k=20(t-1)-EMA_close_k=5(t-1))/precio_medio(t-1)"] = (df_aux["EMA_close_k=20(t)"].shift(1) - df_aux["EMA_close_k=5(t)"].shift(1)) / df_aux["precio_medio"].shift(1)
  df_aux ["(EMA_close_k=20(t-1)-precio_medio(t-1)))/precio_medio(t-1)"] = (df_aux["EMA_close_k=20(t)"].shift(1) - df_aux["precio_medio"].shift(1)) / df_aux["precio_medio"].shift(1)

  df_aux["(EMA_high_k=3(t-1)-EMA_close_k=3(t-1))/precio_medio(t-1)"] = (df_aux["EMA_high_k=3(t)"].shift(1) - df_aux["EMA_close_k=3(t)"].shift(1)) / df_aux["precio_medio"].shift(1)
  df_aux["(EMA_close_k=3(t-1)-EMA_low_k=3(t-1))/precio_medio(t-1)"] = (df_aux["EMA_close_k=3(t)"].shift(1) - df_aux["EMA_low_k=3(t)"].shift(1)) / df_aux["precio_medio"].shift(1)
  df_aux["(EMA_high_k=5(t-1)-EMA_low_k=5(t-1))/precio_medio(t-1)"] = (df_aux["EMA_high_k=5(t)"].shift(1) - df_aux["EMA_low_k=5(t)"].shift(1)) / df_aux["precio_medio"].shift(1)



  # Creación de variables de tendencia:  
  for i in range (1,6):
    df_aux ["movimiento_descendente(t-"+str(i)+")"] = np.where( df_aux ["precio_medio"].shift(i) < df_aux ["precio_medio"].shift(i+1), 1, 0 )
    df_aux ["movimiento_ascendente(t-"+str(i)+")"] = np.where( df_aux ["precio_medio"].shift(i) > df_aux ["precio_medio"].shift(i+1), 1, 0 )


  df_aux ["tendencia_descendente_pequeña"] = np.where( sum([df_aux["movimiento_descendente(t-1)"], df_aux["movimiento_descendente(t-2)"]]) == 2, 1, 0 )
  df_aux ["tendencia_descendente_mediana"] = np.where( sum([df_aux["movimiento_descendente(t-1)"], df_aux["movimiento_descendente(t-2)"],
                                                        df_aux["movimiento_descendente(t-3)"]]) == 3, 1, 0 )
  df_aux ["tendencia_descendente_mediana_intermedia"] = np.where( (sum([df_aux["movimiento_descendente(t-1)"], df_aux["movimiento_descendente(t-2)"],
                                                                   df_aux["movimiento_descendente(t-3)"], df_aux["movimiento_descendente(t-4)"]]) >= 3) &
                                                                  (df_aux["movimiento_descendente(t-1)"] == 1), 1, 0 )
  df_aux ["tendencia_descendente_grande"] = np.where( (sum([df_aux["movimiento_descendente(t-1)"], df_aux["movimiento_descendente(t-2)"],
                                                        df_aux["movimiento_descendente(t-3)"], df_aux["movimiento_descendente(t-4)"],
                                                        df_aux["movimiento_descendente(t-5)"]]) >= 4) & (df_aux["movimiento_descendente(t-1)"] == 1), 1, 0 )

  df_aux ["tendencia_ascendente_pequeña"] = np.where( sum([df_aux["movimiento_ascendente(t-1)"], df_aux["movimiento_ascendente(t-2)"]]) == 2, 1, 0 )
  df_aux ["tendencia_ascendente_mediana"] = np.where( sum([df_aux["movimiento_ascendente(t-1)"], df_aux["movimiento_ascendente(t-2)"],
                                                        df_aux["movimiento_ascendente(t-3)"]]) == 3, 1, 0 )
  df_aux ["tendencia_ascendente_mediana_intermedia"] = np.where( (sum([df_aux["movimiento_ascendente(t-1)"], df_aux["movimiento_ascendente(t-2)"],
                                                                   df_aux["movimiento_ascendente(t-3)"], df_aux["movimiento_ascendente(t-4)"]]) >= 3) &
                                                                  (df_aux["movimiento_ascendente(t-1)"] == 1), 1, 0 )
  df_aux ["tendencia_ascendente_grande"] = np.where( (sum([df_aux["movimiento_ascendente(t-1)"], df_aux["movimiento_ascendente(t-2)"],
                                                        df_aux["movimiento_ascendente(t-3)"], df_aux["movimiento_ascendente(t-4)"],
                                                        df_aux["movimiento_ascendente(t-5)"]]) >= 4) & (df_aux["movimiento_ascendente(t-1)"] == 1), 1, 0 )



  # Creación de Valles y Cimas:

  # Se denomina Valle cuando la cotización disminuye hasta cierto momento donde empieza a subir.
  df_aux ["valle_tipo_1"] = np.where( (sum([df_aux ["movimiento_ascendente(t-1)"] , df_aux["movimiento_ascendente(t-2)"]]) == 2) &
                                      (sum([df_aux ["movimiento_descendente(t-3)"] , df_aux["movimiento_descendente(t-4)"]]) == 2) , 1, 0 ) 
  
  # Se denomina Cima cuando la cotización aumenta hasta cierto momento donde empieza a caer.
  df_aux ["cima_tipo_1"] = np.where( (sum([df_aux ["movimiento_descendente(t-1)"] , df_aux["movimiento_descendente(t-2)"]]) == 2) &
                                     (sum([df_aux ["movimiento_ascendente(t-3)"] , df_aux["movimiento_ascendente(t-4)"]]) == 2) , 1, 0 ) 


  df_aux ["valle_tipo_2"] = np.where( (sum([df_aux ["movimiento_ascendente(t-1)"] , df_aux["movimiento_ascendente(t-2)"]]) == 2) &
                                      (sum([df_aux ["movimiento_descendente(t-3)"] , df_aux["movimiento_descendente(t-4)"],
                                          df_aux["movimiento_descendente(t-5)"]]) >= 2),  1, 0 ) 

  df_aux ["cima_tipo_2"] = np.where( (sum([df_aux ["movimiento_descendente(t-1)"] , df_aux["movimiento_descendente(t-2)"]]) == 2) &
                                     (sum([df_aux ["movimiento_ascendente(t-3)"] , df_aux["movimiento_ascendente(t-4)"],
                                          df_aux["movimiento_ascendente(t-5)"]]) >= 2) , 1, 0 ) 
    

  # Creación de variables a MEDIANO / LARGO plazo:
  añadir_variable_maximo_de_high_sobre_open(df_aux,5)
  añadir_variable_maximo_de_high_sobre_open(df_aux,10)
  añadir_variable_maximo_de_high_sobre_open(df_aux,15)
  añadir_variable_maximo_de_high_sobre_open(df_aux,20)
  añadir_variable_maximo_de_high_sobre_open(df_aux,30)
  añadir_variable_maximo_de_high_sobre_open(df_aux,40)
  añadir_variable_maximo_de_high_sobre_open(df_aux,60)

  añadir_variable_minimo_de_high_sobre_open(df_aux,5)
  añadir_variable_minimo_de_high_sobre_open(df_aux,10)
  añadir_variable_minimo_de_high_sobre_open(df_aux,15)
  añadir_variable_minimo_de_high_sobre_open(df_aux,20)
  añadir_variable_minimo_de_high_sobre_open(df_aux,25)
  añadir_variable_minimo_de_high_sobre_open(df_aux,30)
  añadir_variable_minimo_de_high_sobre_open(df_aux,40)
  añadir_variable_minimo_de_high_sobre_open(df_aux,60)

  añadir_variable_minimo_de_low_sobre_open(df_aux,5)
  añadir_variable_minimo_de_low_sobre_open(df_aux,10)
  añadir_variable_minimo_de_low_sobre_open(df_aux,15)
  añadir_variable_minimo_de_low_sobre_open(df_aux,20)
  añadir_variable_minimo_de_low_sobre_open(df_aux,25)
  añadir_variable_minimo_de_low_sobre_open(df_aux,30)
  añadir_variable_minimo_de_low_sobre_open(df_aux,40)
  añadir_variable_minimo_de_low_sobre_open(df_aux,60)

  añadir_variable_close_mas_X_dias_sobre_open(df_aux,10)
  añadir_variable_close_mas_X_dias_sobre_open(df_aux,15)
  añadir_variable_close_mas_X_dias_sobre_open(df_aux,25)

  lista_de_df.append(df_aux)

data = pd.concat(lista_de_df, axis=0)


tiempo_final = time()  # Para medir el tiempo
tiempo_ejecucion = tiempo_final - tiempo_inicial
print("")
print ('El tiempo de ejecución fue:',tiempo_ejecucion, 'segundos')


In [ ]:
# Creamos más features:

data["variacion_close_(t-2)/variacion_dolar_blue_(t-2)"] = data["close(t-1)/close(t-2)"] / data["dolar_blue(t-1)/dolar_blue(t-2)"]
data["variacion_close_(t-5)/variacion_dolar_blue_(t-5)"] = data["close(t-1)/close(t-5)"] / data["dolar_blue(t-1)/dolar_blue(t-5)"]
data["variacion_close_(t-10)/variacion_dolar_blue_(t-10)"] = data["close(t-1)/close(t-10)"] / data["dolar_blue(t-1)/dolar_blue(t-10)"]
data["variacion_close_(t-20)/variacion_dolar_blue_(t-20)"] = data["close(t-1)/close(t-20)"] / data["dolar_blue(t-1)/dolar_blue(t-20)"]
data["variacion_close_(t-50)/variacion_dolar_blue_(t-50)"] = data["close(t-1)/close(t-50)"] / data["dolar_blue(t-1)/dolar_blue(t-50)"]

In [ ]:
# Creación de variables nominales binarias (0 ó 1) a partir de un valor umbral:

for umbral in (1.01 , 1.015, 1.018, 1.02 , 1.025, 1.03 , 1.035, 1.04):
  añadir_variable_nominal_binaria_mayor_que (data,"high(t)/open(t)",umbral)
  for i in range(1):
        lista_de_df = []
        for accion in data.groupby(['titulo']).groups.keys():
            df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
            df_aux ["umbral_=_"+str(umbral)+"_high(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["umbral_=_"+str(umbral)+"_high(t)/open(t)"].shift(i+1)
            lista_de_df.append(df_aux)
        data = pd.concat(lista_de_df, axis=0)

umbral = 1.015
añadir_variable_nominal_binaria_mayor_que (data,"close(t)/open(t)",umbral)
for i in range(1):
    lista_de_df = []
    for accion in data.groupby(['titulo']).groups.keys():
        df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
        df_aux ["umbral_=_"+str(umbral)+"_close(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["umbral_=_"+str(umbral)+"_close(t)/open(t)"].shift(i+1)
        lista_de_df.append(df_aux)
    data = pd.concat(lista_de_df, axis=0)

umbral = 1.02
añadir_variable_nominal_binaria_mayor_que (data,"close(t)/open(t)",umbral)
for i in range(3):
    lista_de_df = []
    for accion in data.groupby(['titulo']).groups.keys():
        df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
        df_aux ["umbral_=_"+str(umbral)+"_close(t-"+str(i+1)+")/open(t-"+str(i+1)+")"] = df_aux ["umbral_=_"+str(umbral)+"_close(t)/open(t)"].shift(i+1)
        lista_de_df.append(df_aux)
    data = pd.concat(lista_de_df, axis=0)
    

umbral = 1.04
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_5_dias/open(t)",umbral)

umbral = 1.05
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_5_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)

umbral = 1.06
añadir_variable_nominal_binaria_mayor_que (data,"close(t+10)/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"close(t+15)/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)

umbral = 1.08
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_5_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"close(t+10)/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"close(t+15)/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"close(t+25)/open(t)",umbral)

umbral = 1.1
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_20_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_5_dias/open(t)",umbral)

umbral = 1.12
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_10_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_5_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_20_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_40_dias/open(t)",umbral)

umbral = 1.15
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_20_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_30_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_40_dias/open(t)",umbral)

umbral = 1.2
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_20_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_30_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_40_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_60_dias/open(t)",umbral)

umbral = 1.3
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_15_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_40_dias/open(t)",umbral)
añadir_variable_nominal_binaria_mayor_que (data,"maximo_de_high_en_proximos_60_dias/open(t)",umbral)


umbral = 0.95
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )

umbral = 0.92
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_15_dias/open(t)",umbral )

umbral = 0.90
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_30_dias/open(t)",umbral )

umbral = 0.88
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_20_dias/open(t)",umbral )

umbral = 0.85
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_30_dias/open(t)",umbral )

umbral = 0.80
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_40_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_60_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_5_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_10_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_25_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_40_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_low_en_proximos_60_dias/open(t)",umbral )

umbral = 0.75
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_40_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_60_dias/open(t)",umbral )

umbral = 0.70
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_15_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_20_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_30_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_40_dias/open(t)",umbral )
añadir_variable_nominal_binaria_menor_que (data, "minimo_de_high_en_proximos_60_dias/open(t)",umbral )


# Construimos variables umbrales "combinadas", que surgen de que se cumplan 2 condiciones simultáneas:

variable_target_1 = "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.06_maximo_de_high_en_proximos_15_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)

variable_target_1 = "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.06_maximo_de_high_en_proximos_10_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)

variable_target_1 = "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.05_maximo_de_high_en_proximos_5_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)

variable_target_1 = "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.06_maximo_de_high_en_proximos_15_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)

variable_target_1 = "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.06_maximo_de_high_en_proximos_10_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)

variable_target_1 = "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)"
variable_target_2 = "umbral_=_1.05_maximo_de_high_en_proximos_5_dias/open(t)"
data[variable_target_1 + "_&_∼_" + variable_target_2]  = np.where((data[variable_target_1] == 1) & (data[variable_target_2] == 0), 1, 0)


In [142]:
# Creación de variables nominales binarias (0 ó 1) para cada título a través de One Hot Encoding:
data = aplicar_one_hot_encoding_a_un_DataFrame (data, "titulo")

In [143]:
# Creación de variables nominales binarias (0 ó 1) a partir de la información que obtuvimos de los Baselines:

umbral = 1.1
lista_de_df = []

for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
  df_aux ["prediccion_del_baseline"] = np.where( df_aux["high"].shift(1) / df_aux["low"].shift(1) >= umbral, 1, 0)
  lista_de_df.append (df_aux["prediccion_del_baseline"])

data["umbral_=_1.1_high(t-1)/low(t-1)"] = pd.concat(lista_de_df, axis=0) 

umbral = 1.13
lista_de_df = []

for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
  df_aux ["prediccion_del_baseline"] = np.where( df_aux["high"].shift(1) / df_aux["low"].shift(1) >= umbral, 1, 0)
  lista_de_df.append (df_aux["prediccion_del_baseline"])

data["umbral_=_1.13_high(t-1)/low(t-1)"] = pd.concat(lista_de_df, axis=0) 

umbral = 1.15
lista_de_df = []

for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
  df_aux ["prediccion_del_baseline"] = np.where( df_aux["high"].shift(1) / df_aux["low"].shift(1) >= umbral, 1, 0)
  lista_de_df.append (df_aux["prediccion_del_baseline"])

data["umbral_=_1.15_high(t-1)/low(t-1)"] = pd.concat(lista_de_df, axis=0) 

umbral = 1.28
lista_de_df = []

for accion in data.groupby(['titulo']).groups.keys():
  df_aux = data.groupby(['titulo']).get_group(accion).sort_values(by="fecha") 
  df_aux ["prediccion_del_baseline"] = np.where( df_aux["high"].shift(1) / df_aux["low"].shift(1) >= umbral, 1, 0)
  lista_de_df.append (df_aux["prediccion_del_baseline"])

data["umbral_=_1.28_high(t-1)/low(t-1)"] = pd.concat(lista_de_df, axis=0) 


In [ ]:
# Agregamos todos los indicadores técnicos que están incluidos en la librería 'ta' de Python:
data = agregar_todos_los_indicadores_tecnicos_de_la_libreria_ta(data)

In [145]:
# Creamos features simples para identificar las tendencias en la cotización:
data["TA_LIB_trend_psar_down_simplificado"] = np.where( data["TA_LIB_trend_psar_down"]!=0 , 1, 0 )  #Indica con un 1 cuando se está en tendencia bajista, y 0 en caso contrario.
data["TA_LIB_trend_psar_up_simplificado"] = np.where( data["TA_LIB_trend_psar_up"]!=0 , 1, 0 )   #Indica con un 1 cuando se está en tendencia alcista, y 0 en caso contrario.


In [146]:
# Ordenamos el DataFrame por fecha:
data = data.sort_values( by=["fecha"], ascending = True )

## Catálogo de los Tipos de Variables:

In [148]:
# Catalogamos las variables del DataFrame en 4 categorías:
#  - Variables Target Intradiarias  (son variables Dependientes)
#  - Variables Target de Largo Plazo  (son variables Dependientes)
#  - Variables NO utilizadas para entrenar (los modelos NO las tienen en cuenta en ningún aspecto)
#  - Variables Independientes (serán todas las demás variables que no estén especificadas en las categorías anteriores y se catalogarán automáticamente)

variables_no_utilizadas_para_entrenar = {"titulo","fecha_IOL","open","close","high","low","volumen","fecha", "precio_medio",
                                         "dolar_blue", "dolar_ccl", "dolar_blue_compra", "dolar_blue_venta", "dolar_ccl_compra", "dolar_ccl_venta",
                                         "open_pesos", "close_pesos", "low_pesos", "high_pesos",
                                        'SMA_close_k=5(t)', 'EMA_close_k=5(t)', 'EMA_close_k=10(t)', 'EMA_close_k=20(t)',
                                        'EMA_close_k=50(t)', 'EMA_close_k=100(t)', 'EMA_close_k=30(t)', 'EMA_close_k=15(t)',
                                        'EMA_high_k=5(t)', 'EMA_high_k=10(t)', 'EMA_high_k=20(t)', 'EMA_high_k=50(t)',
                                        "SMA_close_k=3(t)","SMA_close_k=5(t)","SMA_close_k=10(t)",
                                        "EMA_close_k=3(t)", "subio_o_bajo_(t)","es_o_no_canal_paralelo_(t-1)", "año","dia","mes",
                                        'SMA_close_k=5(t)/SMA_close_k=5(t-1)', 'EMA_close_k=5(t)/EMA_close_k=5(t-1)',
                                        'EMA_close_k=10(t)/EMA_close_k=10(t-1)','EMA_close_k=20(t)/EMA_close_k=20(t-1)', 
                                        'EMA_high_k=5(t)/EMA_high_k=5(t-1)', 'EMA_high_k=10(t)/EMA_high_k=10(t-1)',
                                        'EMA_high_k=20(t)/EMA_high_k=20(t-1)', 'EMA_high_k=50(t)/EMA_high_k=50(t-1)',
                                        'EMA_close_k=5(t)/EMA_close_k=10(t)', 'EMA_close_k=10(t)/EMA_close_k=20(t)',
                                        'EMA_close_k=20(t)/EMA_close_k=50(t)', 'EMA_high_k=5(t)/EMA_high_k=10(t)',
                                        'EMA_high_k=10(t)/EMA_high_k=20(t)', 'EMA_high_k=20(t)/EMA_high_k=50(t)',
                                        'EMA_close_k=50(t)/EMA_close_k=50(t-1)', "(high(t)+low(t)/2) / EMA_close_k=30(t)",
                                        '(high(t)+low(t)/2) / EMA_close_k=100(t)', "(high(t)+low(t)/2) / EMA_close_k=15(t)",
                                        '(high(t)+low(t)/2) / EMA_close_k=100(t)', "(high(t)+low(t)/2) / EMA_close_k=30(t)",
                                        '(high(t)+low(t)/2) / EMA_close_k=15(t)', "EMA_close_k=40(t)", "EMA_high_k=3(t)",
                                         "EMA_high_k=5(t)", "EMA_high_k=10(t)", "EMA_high_k=2(t)", "EMA_close_k=2(t)", "EMA_low_k=5(t)",
                                         "EMA_open_k=5(t)", "EMA_open_k=6(t)", "EMA_close_k=25(t)", "EMA_low_k=10(t)", "EMA_low_k=3(t)",
                                         "EMA_open_k=10(t)", "EMA_open_k=20(t)", "EMA_low_k=20(t)", "EMA_high_k=20(t)",
                                         "EMA_dolar_blue_k=20(t)", "EMA_dolar_ccl_k=20(t)", "EMA_dolar_blue_k=2(t)",
                                         "EMA_dolar_ccl_k=3(t)", "EMA_dolar_blue_k=3(t)", "EMA_dolar_ccl_k=2(t)",
                                         "EMA_dolar_ccl_k=10(t)", "EMA_dolar_blue_k=10(t)", "EMA_dolar_blue_k=40(t)", "EMA_dolar_ccl_k=40(t)",
                                         "EMA_dolar_blue_k=50(t)", "EMA_dolar_ccl_k=50(t)", "EMA_dolar_blue_k=5(t)", "EMA_dolar_ccl_k=5(t)",
                                         "EMA_dolar_ccl_k=25(t)", "EMA_dolar_blue_k=25(t)", "EMA_dolar_ccl_k=15(t)", "EMA_dolar_blue_k=15(t)",
                                         "EMA_dolar_blue_k=30(t)", "EMA_dolar_ccl_k=30(t)"
                                         }

variables_target_intradiarias = {"close(t)/close(t-1)","high(t)/high(t-1)","low(t)/low(t-1)","volumen(t)/volumen(t-1)",
                                'high(t)/open(t)', 'close(t)/open(t)', 'low(t)/open(t)', 'high(t)/low(t)', 'high(t)/low(t-1)',
                                'high(t)/close(t)', 'high(t)/close(t-1)', "umbral_=_1.03_high(t)/open(t)", "umbral_=_1.02_close(t)/open(t)",
                                'umbral_=_1.015_close(t)/open(t)', "umbral_=_1.013_close(t)/open(t)", "umbral_=_1.01_high(t)/open(t)",
                                 "umbral_=_1.015_high(t)/open(t)", "umbral_=_1.02_high(t)/open(t)", "umbral_=_1.025_high(t)/open(t)",
                                 "umbral_=_1.035_high(t)/open(t)", "umbral_=_1.04_high(t)/open(t)", "umbral_=_1.018_high(t)/open(t)"                            
                                 }

variables_target_de_largo_plazo = {"low(t+3)/open(t)", "minimo_de_low_en_proximos_3_dias/open(t)",
                                  "maximo_de_high_en_proximos_20_dias/open(t)", "umbral_=_1.1_maximo_de_high_en_proximos_10_dias/open(t)",
                                  "maximo_de_high_en_proximos_40_dias/open(t)","maximo_de_high_en_proximos_5_dias/open(t)",
                                  "maximo_de_high_en_proximos_10_dias/open(t)", "umbral_=_1.08_maximo_de_high_en_proximos_10_dias/open(t)",
                                  "umbral_=_1.1_maximo_de_high_en_proximos_5_dias/open(t)", "umbral_=_1.08_maximo_de_high_en_proximos_5_dias/open(t)",
                                  "umbral_=_1.12_maximo_de_high_en_proximos_5_dias/open(t)", "maximo_de_high_en_proximos_15_dias/open(t)",
                                  "umbral_=_1.12_maximo_de_high_en_proximos_10_dias/open(t)", "umbral_=_1.1_maximo_de_high_en_proximos_15_dias/open(t)",
                                  "umbral_=_1.1_maximo_de_high_en_proximos_25_dias/open(t)", "umbral_=_1.12_maximo_de_high_en_proximos_15_dias/open(t)",
                                  "umbral_=_1.08_maximo_de_high_en_proximos_15_dias/open(t)", "umbral_=_1.06_close(t+15)/open(t)", 
                                  "umbral_=_1.08_close(t+15)/open(t)", "umbral_=_1.06_close(t+10)/open(t)", "umbral_=_1.08_close(t+10)/open(t)",
                                  "umbral_=_1.08_close(t+25)/open(t)", "close(t+45)/open(t)", "close(t+25)/open(t)", "close(t+15)/open(t)",
                                  "close(t+10)/open(t)", "close(t+5)/open(t)", "umbral_=_1.06_maximo_de_high_en_proximos_15_dias/open(t)",
                                  "umbral_=_1.06_maximo_de_high_en_proximos_10_dias/open(t)", "umbral_=_1.05_maximo_de_high_en_proximos_10_dias/open(t)",
                                  "umbral_=_1.05_maximo_de_high_en_proximos_15_dias/open(t)", "umbral_=_1.04_maximo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_1.1_maximo_de_high_en_proximos_20_dias/open(t)", "umbral_=_1.04_maximo_de_high_en_proximos_5_dias/open(t)",
                                   "umbral_=_1.05_maximo_de_high_en_proximos_5_dias/open(t)", "maximo_de_high_en_proximos_30_dias/open(t)",
                                   "maximo_de_high_en_proximos_40_dias/open(t)", "maximo_de_high_en_proximos_60_dias/open(t)",
                                   "umbral_=_1.3_maximo_de_high_en_proximos_60_dias/open(t)", "umbral_=_1.2_maximo_de_high_en_proximos_60_dias/open(t)",
                                   "umbral_=_1.15_maximo_de_high_en_proximos_20_dias/open(t)", "umbral_=_1.15_maximo_de_high_en_proximos_40_dias/open(t)",
                                   "umbral_=_1.3_maximo_de_high_en_proximos_40_dias/open(t)", "umbral_=_1.12_maximo_de_high_en_proximos_40_dias/open(t)",
                                   "umbral_=_1.12_maximo_de_high_en_proximos_20_dias/open(t)", "umbral_=_1.2_maximo_de_high_en_proximos_30_dias/open(t)",
                                   "umbral_=_1.2_maximo_de_high_en_proximos_40_dias/open(t)", "umbral_=_1.15_maximo_de_high_en_proximos_30_dias/open(t)",
                                   "umbral_=_1.2_maximo_de_high_en_proximos_20_dias/open(t)", "minimo_de_high_en_proximos_60_dias/open(t)",
                                   "minimo_de_high_en_proximos_30_dias/open(t)", "minimo_de_high_en_proximos_20_dias/open(t)",
                                   "minimo_de_high_en_proximos_10_dias/open(t)", "minimo_de_high_en_proximos_5_dias/open(t)",
                                   "minimo_de_high_en_proximos_15_dias/open(t)", "umbral_=_0.75_minimo_de_high_en_proximos_15_dias/open(t)",
                                   "umbral_=_0.75_minimo_de_high_en_proximos_20_dias/open(t)", "umbral_=_0.75_minimo_de_high_en_proximos_30_dias/open(t)",
                                   "umbral_=_0.75_minimo_de_high_en_proximos_40_dias/open(t)", "umbral_=_0.75_minimo_de_high_en_proximos_60_dias/open(t)",
                                   "umbral_=_0.7_minimo_de_high_en_proximos_15_dias/open(t)", "umbral_=_0.7_minimo_de_high_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.7_minimo_de_high_en_proximos_30_dias/open(t)", "umbral_=_0.7_minimo_de_high_en_proximos_40_dias/open(t)",
                                   "umbral_=_0.7_minimo_de_high_en_proximos_60_dias/open(t)", "umbral_=_0.85_minimo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.85_minimo_de_high_en_proximos_15_dias/open(t)", "umbral_=_0.85_minimo_de_high_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.85_minimo_de_high_en_proximos_5_dias/open(t)", "umbral_=_0.8_minimo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_high_en_proximos_15_dias/open(t)", "umbral_=_0.8_minimo_de_high_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_high_en_proximos_30_dias/open(t)", "umbral_=_0.8_minimo_de_high_en_proximos_40_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_high_en_proximos_60_dias/open(t)", "umbral_=_0.95_minimo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.95_minimo_de_high_en_proximos_5_dias/open(t)", "umbral_=_0.9_minimo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_high_en_proximos_15_dias/open(t)", "umbral_=_0.9_minimo_de_high_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_high_en_proximos_5_dias/open(t)", "minimo_de_high_en_proximos_40_dias/open(t)",
                                   "umbral_=_0.85_minimo_de_high_en_proximos_30_dias/open(t)", "umbral_=_0.9_minimo_de_low_en_proximos_15_dias/open(t)",
                                   "minimo_de_low_en_proximos_30_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_30_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_high_en_proximos_5_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_10_dias/open(t)",
                                   "minimo_de_low_en_proximos_5_dias/open(t)", "umbral_=_0.85_minimo_de_low_en_proximos_25_dias/open(t)",
                                   "minimo_de_low_en_proximos_25_dias/open(t)", "umbral_=_0.95_minimo_de_low_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.88_minimo_de_low_en_proximos_15_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.85_minimo_de_low_en_proximos_30_dias/open(t)", "umbral_=_0.8_minimo_de_high_en_proximos_25_dias/open(t)",
                                   "minimo_de_low_en_proximos_20_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_5_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_high_en_proximos_25_dias/open(t)", "umbral_=_0.85_minimo_de_low_en_proximos_5_dias/open(t)",
                                   "minimo_de_low_en_proximos_10_dias/open(t)", "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.85_minimo_de_low_en_proximos_10_dias/open(t)", "umbral_=_0.9_minimo_de_low_en_proximos_30_dias/open(t)",
                                   "umbral_=_0.95_minimo_de_low_en_proximos_5_dias/open(t)", "umbral_=_0.9_minimo_de_low_en_proximos_20_dias/open(t)",
                                   "minimo_de_low_en_proximos_60_dias/open(t)", "umbral_=_0.9_minimo_de_low_en_proximos_5_dias/open(t)",
                                   "umbral_=_0.88_minimo_de_low_en_proximos_5_dias/open(t)", "umbral_=_0.85_minimo_de_low_en_proximos_20_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_low_en_proximos_25_dias/open(t)", "umbral_=_0.85_minimo_de_low_en_proximos_15_dias/open(t)",
                                   "minimo_de_low_en_proximos_40_dias/open(t)", "umbral_=_0.88_minimo_de_low_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_low_en_proximos_25_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_15_dias/open(t)",
                                   "umbral_=_0.88_minimo_de_low_en_proximos_20_dias/open(t)", "umbral_=_0.85_minimo_de_high_en_proximos_25_dias/open(t)",
                                   "umbral_=_0.8_minimo_de_low_en_proximos_60_dias/open(t)", "minimo_de_high_en_proximos_25_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_high_en_proximos_30_dias/open(t)", "umbral_=_0.8_minimo_de_low_en_proximos_40_dias/open(t)",
                                   "minimo_de_low_en_proximos_15_dias/open(t)", "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.92_minimo_de_low_en_proximos_5_dias/open(t)", "umbral_=_0.92_minimo_de_low_en_proximos_15_dias/open(t)",
                                   "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.06_maximo_de_high_en_proximos_15_dias/open(t)",
                                   "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.05_maximo_de_high_en_proximos_5_dias/open(t)",
                                   "umbral_=_0.92_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.06_maximo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.06_maximo_de_high_en_proximos_10_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.06_maximo_de_high_en_proximos_15_dias/open(t)",
                                   "umbral_=_0.9_minimo_de_low_en_proximos_10_dias/open(t)_&_∼_umbral_=_1.05_maximo_de_high_en_proximos_5_dias/open(t)",
                                   "umbral_=_1.3_maximo_de_high_en_proximos_15_dias/open(t)"
                                  } 

# El set "variables_dependientes" sale de unir los conjuntos de las Variables Target Intradiarias y las Variables Target de Largo Plazo:
variables_dependientes = variables_target_intradiarias.union(variables_target_de_largo_plazo)

variables_todas = set(data.columns)

# El set "variables_independientes" se obtiene de agarrar todas las variables y restar los conjuntos de las Variables NO Utilizadas para Entrenar y 
# las Variables Dependientes:
variables_independientes = variables_todas - variables_no_utilizadas_para_entrenar.union(variables_dependientes)

In [149]:
# Chequeamos que no haya variables futuras en el set 'variables_independientes':
chequeo_ausencia_de_variables_futuras_en_las_variables_independientes(variables_independientes)

In [150]:
# Exportamos las listas de variables en formato 'pickle' para que puedan ser leídas por otros archivos de código:

with open('variables_target_intradiarias.pickle', 'wb') as archivo:
    pickle.dump(variables_target_intradiarias, archivo)

with open('variables_target_de_largo_plazo.pickle', 'wb') as archivo:
    pickle.dump(variables_target_de_largo_plazo, archivo)

with open('variables_independientes.pickle', 'wb') as archivo:
    pickle.dump(variables_independientes, archivo)
    
with open('variables_dependientes.pickle', 'wb') as archivo:
    pickle.dump(variables_dependientes, archivo)
    

## Filtrado por Fecha y Tratamiento de NaNs:

In [151]:
# Nos quedamos únicamente con los datos entre los días 2010-07-01 y 2020-06-30:
data = data[data.fecha >= pd.to_datetime(datetime.date(2010, 7, 1))]
data = data[data.fecha < pd.to_datetime(datetime.date(2022, 7, 1))]

In [ ]:
# Tratamiento de los NaN:

# Identificamos la cantidad de filas que tienen valores NaN:
cantidad_de_filas_con_NaN = len(data[data.isnull().any(axis=1)])
print("La cantidad de filas que tienen valores NaN es:", cantidad_de_filas_con_NaN)

if cantidad_de_filas_con_NaN != 0:
  
  # Eliminamos del DataFrame aquellas filas con valores NaN:
  lista_de_index_con_NaN = data[data.isnull().any(axis=1)].index
  data = data.drop(lista_de_index_con_NaN, axis=0)
  print("Estas filas han sido eliminadas del DataFrame.")


In [153]:
data_train_y_test = data[data.fecha < pd.to_datetime(datetime.date(2020, 7, 1))].copy()

# Ordenamos por "fecha" y por "titulo":
data_train_y_test = data_train_y_test.sort_values(by=["fecha", "titulo"])

In [154]:
data_periodo_subsiguiente = data[data.fecha >= pd.to_datetime(datetime.date(2020, 7, 1))].copy()

# Ordenamos por "fecha" y por "titulo":
data_periodo_subsiguiente = data_periodo_subsiguiente.sort_values(by=["fecha", "titulo"])

## Exportación:

In [155]:
ruta_de_guardado = ".\\2. Preprocesado de Datos\\"
nombre_del_archivo_train_y_test = 'Datos_Completos_Preprocesados'
nombre_del_archivo_periodo_subsiguiente = 'Datos_Completos_Preprocesados_Periodo_subsiguiente'
extension_del_archivo = '.csv'

# Exportamos:
data_train_y_test.to_csv( (ruta_de_guardado + nombre_del_archivo_train_y_test + agregado_al_nombre_del_archivo + extension_del_archivo), index = False )
data_periodo_subsiguiente.to_csv( (ruta_de_guardado + nombre_del_archivo_periodo_subsiguiente + agregado_al_nombre_del_archivo + extension_del_archivo), index = False )